In [63]:
## The major processes of NLP befor feeding the data to an ML models are :
# 1. Text Cleaning --- lowercase, remove spacial character's and url's, removing stopwords
# 2. Tokenization  --- splitting the text into word.
# 3. Normalization --- applying stemming or lemmatization to get the root word of the word (removing suffix and preffix)
# 4. Vectorization --- converting the text data into a vertor array that can be feeded to an ML model.

# LET'S START THE PROCESS STEP BY STEP

In [188]:
# Importing the all nesessary librarie's here :
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from datasets import load_dataset
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models import Word2Vec
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, confusion_matrix, f1_score
from sklearn.ensemble import RandomForestClassifier
import re 
import nltk
import warnings

In [65]:
warnings.filterwarnings('ignore')

In [66]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\emonp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [67]:
stopwords = set(stopwords.words('english'))
print('not' in stopwords)   # By default word 'not' is present in stopwords.

True


In [68]:
stopwords.remove('not')

In [69]:
                                                         ###Importing the data here and analysing the data : ###
data = pd.read_csv('tweets.csv')
data.columns=['sentiment','ID','DataTime','Query','Name','comments']

In [70]:
data.head()

,sentiment,ID,DataTime,Query,Name,comments
0,0,1984664187,Sun May 31 15:33:25 PDT 2009,NO_QUERY,XeBeans,another WONDERFUL weekend! i almost hate to go...
1,0,2187830458,Mon Jun 15 20:52:20 PDT 2009,NO_QUERY,ACTinglikeamama,belly pushing against jeans - only three month...
2,0,1992717155,Mon Jun 01 09:12:29 PDT 2009,NO_QUERY,scarletsighting,missing London. with all honesty.
3,4,1792666808,Wed May 13 23:50:50 PDT 2009,NO_QUERY,sambirchall,Wow!!! I can go on twitwer on my iPod that's ...
4,4,1695580504,Mon May 04 06:28:50 PDT 2009,NO_QUERY,SaucyChp,@sweetp0tatos thanks chuck


<!-- So we can see the data is along with ['sentiment','ID','DataTime','Query','Name','comments'] these columns so, now we will take only the sentiment column and comments column and start with
the preprocessing technique
Before start preprocessing we have to check the dataset condition and the distribution of the data  -->

In [71]:
data=data.drop(['ID','DataTime',"Query",'Name'],axis=1)
data.head()

,sentiment,comments
0,0,another WONDERFUL weekend! i almost hate to go...
1,0,belly pushing against jeans - only three month...
2,0,missing London. with all honesty.
3,4,Wow!!! I can go on twitwer on my iPod that's ...
4,4,@sweetp0tatos thanks chuck


In [72]:
data.isnull().sum() ## Finding the missing values

sentiment    0
comments     0
dtype: int64

In [73]:
data['sentiment'].value_counts()

sentiment
0    799597
4    799115
Name: count, dtype: int64

In [74]:
799597-799115

482

### we can see there is difference of 482 sentimentsIn this case we actually don't need to apply resampling because the differece is less than 5% means it's not affecting the data.We use Smote to generate synthetic majority data.we use Oversampling to duplicate the minority data.we use Undersampling to reduce majority class data.By the way we can use SMOTE or Undersampling to equal the data for better performance.

In [75]:
# Now we need to set the value of sentiment column to 0 and 1 because it's an classification problem and we must give the model with binary classification formate
# Otherwise model will get confused.

# Let's do it.

data['sentiment'] = data['sentiment'].replace({4:1}) ## replace 4 with 1's
data.head()

,sentiment,comments
0,0,another WONDERFUL weekend! i almost hate to go...
1,0,belly pushing against jeans - only three month...
2,0,missing London. with all honesty.
3,1,Wow!!! I can go on twitwer on my iPod that's ...
4,1,@sweetp0tatos thanks chuck


In [76]:
data['sentiment'].value_counts()

sentiment
0    799597
1    799115
Name: count, dtype: int64

In [77]:
data=data.loc[np.random.choice(data.index, size=5000, replace=False)] # so i take half of the data for faster training

In [78]:
print(data.shape[0])
data['sentiment'].value_counts()

5000


sentiment
1    2521
0    2479
Name: count, dtype: int64

In [79]:
X = data['comments']
y= data['sentiment']

In [80]:
X.shape

(5000,)

In [81]:
y.shape

(5000,)

In [82]:
# Applying Text cleaning, Tokenization and lemmatization using of post tags with respect to noun,verb,adjective and adverb
corpus = []
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
lemma  = WordNetLemmatizer()
for i in tqdm(X):  
    cleaned_text = re.sub(r'[^a-zA-Z]', ' ', i).lower()  
    words = cleaned_text.split()  
    pos_tags = pos_tag(words)  
    lemma_words = ' '.join(lemma.lemmatize(word, get_wordnet_pos(tag)) for word, tag in pos_tags)
    corpus.append(lemma_words)

100%|██████████| 5000/5000 [00:02<00:00, 1836.45it/s]


In [83]:
corpus

['netdep great thanks i will check these out',
 'nicoleelaynee it didnt work on my phone try bbming it to me and tell hannah that i make her dinner',
 'be downstairs watch a movie i m not tell you which one it be it s proper geeky but i love it',
 'awgnasuha you have one follow me',
 'blairebachman don t be nervous you be go to be great how s life my sweet friend i miss you be home in less than week',
 'missshoko rdj dianakappler yeah but i ve only mn leave before i get to uni',
 'now remove excess glint from my over exuberant flame artist less be more dude',
 'amandacole hrm not really there s one song i like and it s just instrumental',
 'watch taylor at nbc i miss half',
 'sorry fan i have be in canada do research d and brady have be in dallas texas sorry about no twitter long day news comin up',
 'come back from power you guess it traaaffic',
 'i need a hair cut',
 'jennypoynter yeah lol iv still get thankss how do you get on with history',
 'youngq be the opening song quot full se

In [84]:
data['clean_comment']=corpus # Create a new column with the clean data

In [85]:
data.head(20)

,sentiment,comments,clean_comment
1422302,1,"@NetDep Great, thanks. I will check these out.",netdep great thanks i will check these out
114199,0,@nicoleelaynee it didnt work on my phone try ...,nicoleelaynee it didnt work on my phone try bb...
208866,0,Is downstairs watching a movie. I'm not tellin...,be downstairs watch a movie i m not tell you w...
143011,1,@awgnasuha You have one??? Follow me!,awgnasuha you have one follow me
399415,1,@blairebachman Don't be nervous! You are going...,blairebachman don t be nervous you be go to be...
216474,0,@MissShoko_rdj @DianaKappler yeah but I've onl...,missshoko rdj dianakappler yeah but i ve only ...
493675,1,Now removing excess glints from my over-exuber...,now remove excess glint from my over exuberant...
1130590,1,@amandacole1986 Hrm not really. There's one so...,amandacole hrm not really there s one song i l...
781370,0,Watching Taylor at NBC! I missed half,watch taylor at nbc i miss half
287700,0,Sorry FANS I have been in Canada doing resear...,sorry fan i have be in canada do research d an...


In [86]:
X = data['clean_comment']
y = data['sentiment']

## Now i will apply the following concept TF-IDF, Word2Vec and BOW.

In [87]:
# Applying TF-IDF

tfidf = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf.fit_transform(X)
X_tfidf

<5000x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 53991 stored elements in Compressed Sparse Row format>

In [88]:
# So we get an sparse matrix and we need to convert it to array. But we will do it latter.

In [89]:
# Applying BOW

bow = CountVectorizer(max_features=5000)
X_bow = bow.fit_transform(X)
X_bow

<5000x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 53991 stored elements in Compressed Sparse Row format>

In [90]:
X_bow

<5000x5000 sparse matrix of type '<class 'numpy.int64'>'
	with 53991 stored elements in Compressed Sparse Row format>

In [91]:
# Creative word2vec model  
X = data['clean_comment'].apply(lambda x: x.split()).tolist() # creative list of list 
word2vec = Word2Vec(sentences=X, vector_size=300, window=5, min_count=2, workers=4)
word2vec.train(X, total_examples=len(X), epochs=10)

(450175, 691080)

In [92]:
def get_sentence_vector(sentence, model, vector_size=300): 
    words = sentence.split()  # Ensure sentence is tokenized
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(vector_size)

In [93]:
vec_x = np.array([get_sentence_vector(" ".join(sentence), word2vec, vector_size=300) for sentence in X])

In [94]:
vec_x

array([[ 0.02190764,  0.15965751,  0.03775242, ..., -0.06516907,
         0.23455444, -0.24734391],
       [ 0.03295293,  0.18859465, -0.01115055, ..., -0.05138911,
         0.22778262, -0.27119499],
       [-0.02723176,  0.26126352,  0.00758569, ..., -0.07832722,
         0.22574833, -0.31147298],
       ...,
       [-0.0131227 ,  0.25852156,  0.11438093, ..., -0.16232577,
         0.16278003, -0.24258213],
       [ 0.03469567,  0.06403918,  0.00341142, ..., -0.06495466,
         0.1359873 , -0.14644304],
       [ 0.05265597,  0.19019492, -0.0102092 , ..., -0.02925663,
         0.18434373, -0.24662268]])

In [95]:
# Initiate the SMOTE 
smote = SMOTE(random_state=42)

In [96]:
# Split the data into train and test
#TF-IDF
X_tfidf_train,X_tfidf_test,y_tfidf_train,y_tfidf_test = train_test_split(X_tfidf,y,test_size=0.2,random_state=42)

#BOW
X_bow_train,X_bow_test,y_bow_train,y_bow_test = train_test_split(X_bow,y,test_size=0.2,random_state=42)

#Word2Vec
X_vec_train,X_vec_test,y_vec_train,y_vec_test = train_test_split(vec_x,y,test_size=0.2,random_state=42)

In [97]:
print(X_tfidf_train.shape,X_tfidf_test.shape)
print(y_tfidf_train.shape,y_tfidf_test.shape)
# print(X_bow_train.shape,y_train.shape)
# print(X_vec_train.shape,y_train.shape)

(4000, 5000) (1000, 5000)
(4000,) (1000,)


In [98]:
# Now applying the SMOTE for all the models
X_smote_tfidf,y_smote_tfidf = smote.fit_resample(X_tfidf_train,y_tfidf_train)


In [99]:
X_smote_bow,y_smote_bow = smote.fit_resample(X_bow_train,y_bow_train)


In [100]:
X_smote_vec,y_smote_vec = smote.fit_resample(X_vec_train,y_vec_train)

In [101]:
## Now apply the x and y to the ML models. I will apply on (SVM and Logistic Regression)

svm_bow = SVC(probability=True)
svm_tfidf = SVC(probability=True)
svm_vec = SVC(probability=True)
clf_bow = LogisticRegression()
clf_tfidf = LogisticRegression()
clf_vec = LogisticRegression()

In [102]:
# Applying standard scaling for word2vec model 
scaler = StandardScaler()
X_vec_train = scaler.fit_transform(X_vec_train)
X_vec_test = scaler.transform(X_vec_test)

In [103]:
svm_bow.fit(X_bow_train,y_bow_train)


SVC(probability=True)

In [119]:
svm_tfidf.fit(X_tfidf_train,y_tfidf_train)


SVC(probability=True)

In [120]:
svm_vec.fit(X_vec_train,y_vec_train)

SVC(probability=True)

In [121]:
clf_bow.fit(X_bow_train,y_bow_train)

LogisticRegression()

In [122]:
clf_tfidf.fit(X_tfidf_train,y_tfidf_train)

LogisticRegression()

In [108]:
clf_vec.fit(X_vec_train,y_vec_train)

LogisticRegression()

In [147]:
# Though it's an classification problem so i will apply precision to check false positive and the other 3 parameters 

In [148]:
# Predicting on the test dataset 
# SVM 
pred_bow =svm_bow.predict(X_bow_test)

In [149]:
pred_tfidf=svm_tfidf.predict(X_tfidf_test)


In [150]:
print(X_vec_test.shape,X_vec_train.shape)

(1000, 300) (4000, 300)


In [151]:
pred_vec = svm_vec.predict(X_vec_test)

In [159]:
# LogisticRegression
pred_l_bow=clf_bow.predict(X_bow_test)
pred_l_tfidf=clf_tfidf.predict(X_tfidf_test)
pred_l_vec=clf_vec.predict(X_vec_test)

In [190]:
# Now i will apply the F1 score to get the combination of precision and accuracy
svm_pre_bow = f1_score(pred_bow,y_bow_test,average='binary')
svm_pre_tfidf = f1_score(pred_tfidf,y_tfidf_test,average='binary')
svm_pre_vec = f1_score(pred_vec,y_vec_test,average='binary')

In [191]:
print('The F1_score for BOW is: ',svm_pre_bow)
print('The F1_score for TF-IDF is: ',svm_pre_tfidf)
print('The F1_score for Word2Vec is: ',svm_pre_vec)

The F1_score for BOW is:  0.744721689059501
The F1_score for TF-IDF is:  0.7646493756003843
The F1_score for Word2Vec is:  0.6731707317073171


In [189]:
# Now i will apply the confusion metrics to see the actual performance on the test data
svm_con_bow = confusion_matrix(pred_bow,y_bow_test)
svm_con_tfidf = confusion_matrix(pred_tfidf,y_tfidf_test)
svm_con_vec = confusion_matrix(pred_vec,y_vec_test)

In [156]:
print('The confusion Metrics for BOW is: \n',svm_con_bow)
print('The confusion Metrics for TF-IDF is: \n',svm_con_tfidf)
print('The confusion Metrics for Word2Vec is: \n',svm_con_vec)

The confusion Metrics for BOW is: 
 [[346 143]
 [123 388]]
The confusion Metrics for TF-IDF is: 
 [[357 133]
 [112 398]]
The confusion Metrics for Word2Vec is: 
 [[320 186]
 [149 345]]


In [157]:
TN, FP, FN, TP=svm_con_bow.ravel()

In [158]:
FN

123

SO we can see our svm model get 123 of False Negative which values are actual positive but predicted negative which is risky for a model like deseas classification.

In [160]:
clf_con_bow = confusion_matrix(pred_l_bow,y_bow_test)
clf_con_tfidf = confusion_matrix(pred_l_tfidf,y_tfidf_test)
clf_con_vec = confusion_matrix(pred_l_vec,y_vec_test)

In [161]:
print('The confusion Metrics for BOW is: \n',clf_con_bow)
print('The confusion Metrics for TF-IDF is: \n',clf_con_tfidf)
print('The confusion Metrics for Word2Vec is: \n',clf_con_vec)

The confusion Metrics for BOW is: 
 [[352 140]
 [117 391]]
The confusion Metrics for TF-IDF is: 
 [[359 143]
 [110 388]]
The confusion Metrics for Word2Vec is: 
 [[345 176]
 [124 355]]


In [ ]:
# Now i will apply random forest here with the help of gridsearchcv for the best parameter finding.
# BOW(Bag OF Words)
random = RandomForestClassifier(random_state=42)
param = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy']
}
grid_search = GridSearchCV(estimator=random, param_grid=param, 
                           cv=2, scoring='f1', n_jobs=-1, verbose=2)
grid_search.fit(X_bow_train, y_bow_train)



Fitting 2 folds for each of 648 candidates, totalling 1296 fits


GridSearchCV(cv=2, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [10, 20, None],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 5],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='f1', verbose=2)

In [ ]:
# TF-IDF(Term Frequency - Inverce Term Frequency)
random = RandomForestClassifier(random_state=42)
param = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy']
}
grid_search_tf = GridSearchCV(estimator=random, param_grid=param, 
                           cv=2, scoring='f1', n_jobs=-1, verbose=2)


for _ in tqdm(range(len(grid_search.cv_results_["params"]))):
   grid_search.fit(X_tfidf_train, y_tfidf_train)

Fitting 2 folds for each of 648 candidates, totalling 1296 fits


100%|██████████| 648/648 [00:00<00:00, 648665.63it/s]


In [171]:
# Word2Vec (Word To Vector)
random = RandomForestClassifier(random_state=42)
param = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'max_features': ['sqrt', 'log2'],
    'bootstrap': [True, False],
    'criterion': ['gini', 'entropy']
}
grid_search_vec = GridSearchCV(estimator=random, param_grid=param, 
                           cv=2, scoring='f1', n_jobs=-1, verbose=2)
grid_search.fit(X_vec_train, y_vec_train)

Fitting 2 folds for each of 648 candidates, totalling 1296 fits


GridSearchCV(cv=2, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [10, 20, None],
                         'max_features': ['sqrt', 'log2'],
                         'min_samples_leaf': [1, 2, 5],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='f1', verbose=2)

In [176]:
grid_search.best_params_

{'bootstrap': True,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'n_estimators': 100}

So this are the best index for random forest with Bow 

In [177]:
# I will train only one for random because it's takes lots of times 
random_forest = RandomForestClassifier(bootstrap= True,
 criterion= 'gini',
 max_depth= None,
 max_features= 'sqrt',
 min_samples_leaf= 1,
 min_samples_split= 10,
 n_estimators= 100,
 random_state=42
 )

In [178]:
random_forest.fit(X_bow_train,y_bow_train)

RandomForestClassifier(min_samples_split=10, random_state=42)

In [179]:
pred_rand = random_forest.predict(X_bow_test)

In [ ]:
results = np.column_stack((pred_rand, y_bow_test)) 
results


array([[1, 1],
       [0, 0],
       [0, 0],
       ...,
       [0, 1],
       [1, 1],
       [0, 0]], dtype=int64)

In [192]:
# QUESTION ANSWERING 

In [ ]:
# *** question 1 : How does each architecture handle CNN & LSTM sequential text data?

# Answer :

"""1. Convolutional Neural Networks (CNNs)
CNNs are primarily designed to capture local patterns and spatial hierarchies in data. When applied to sequential text data:
CNN Have 3 layer of computation which are , Convolution layer, pooling layer and flatten layer. CNN are useful to use in Image generation or 
classification.
Text data is typically converted into numerical representations, such as word embeddings (Word2Vec,TFIDF) or tokenized sequences.
CNNs are effective for tasks like text classification, sentiment analysis, and short-text understanding, where local patterns are more important than long-range dependencies.

2. Long Short-Term Memory (LSTM) Networks
LSTMs are a type of Recurrent Neural Network (RNN) specifically designed to handle sequential data and long-range dependencies.
LSTMs is an advance form of RNN where the RNN can't handle the sequense, Symantic meaning and it have vanishing gradiant descent problem.
where LSTM can handle the sequential data with long term memory which can remember previous weights. 
LSTM wroks with 3 gates:
1. Input gate
2. Output gate
3. Forget gate

input and output gate basically handles the input and output, where forget get handles to forget any weights. It allows to capture entire sequens."""

"1. Convolutional Neural Networks (CNNs)\nCNNs are primarily designed to capture local patterns and spatial hierarchies in data. When applied to sequential text data:\nCNN Have 3 layer of computation which are , Convolution layer, pooling layer and flatten layer. CNN are useful to use in Image generation or \nclassification.\nText data is typically converted into numerical representations, such as word embeddings (Word2Vec,TFIDF) or tokenized sequences.\nCNNs are effective for tasks like text classification, sentiment analysis, and short-text understanding, where local patterns are more important than long-range dependencies.\n\n2. Long Short-Term Memory (LSTM) Networks\nLSTMs are a type of Recurrent Neural Network (RNN) specifically designed to handle sequential data and long-range dependencies.\nLSTMs is an advance form of RNN where the RNN can't handle the sequense, Symantic meaning and it have vanishing gradiant descent problem.\nwhere LSTM can handle the sequential data with long 

In [ ]:
# Question 2 : Which one would you choose for short tweets vs long text reviews, and why? How does BERT handle contextual word meanings compared to Word2Vec or TF-IDF?

# Answer :

'''In order of choosing a model with short tweets and long text reviews, I will choose Word2Vec for short tweets and BERT for long text reviews.

Reason Word2Vec : 
Word2vec is an technique where a single word compare to 300 word and calculate the score to make it unique then the other words. In google word2vec model 
we have 300 feature extraction and by comparing a single word with 300 word we get the symantic means of that perticular word but the problem is we don't get
positional encoding of a word also can't capture the sequence of the sentence. Although we don't required to get the sequential and positional values in text classification.
we only just nead to understand the context of a sentence.

Reason BERT:
In order to capture the sequential data and positional value of a word to get the output seq2seq we need to apply BERT or Transformers for sequential text, model like text generation.
For that the word will append sequentially with a symantic meaning.

'''

"In order of choosing a model with short tweets and long text reviews, I will choose Word2Vec for short tweets and BERT for long text reviews.\n\nReason Word2Vec : \nWord2vec is an technique where a single word compare to 300 word and calculate the score to make it unique then the other words. In google word2vec model \nwe have 300 feature extraction and by comparing a single word with 300 word we get the symantic means of that perticular word but the problem is we don't get\npositional encoding of a word also can't capture the sequence of the sentence. Although we don't required to get the sequential and positional values in text classification.\nwe only just nead to understand the context of a sentence.\n\nReason BERT:\n\n\n\n\n"